In [1]:
# run this same experiment again but instead of 2,0,0 -> 0,0,2 have 2,0,0 -> 0,1,1 this would be cool 
# declaring all dependancies    
import qutip as qt
import numpy as np
from quantum_logical.pulsesim import QuantumSystem, Pulse
from quantum_logical.pulsesim.mode import QubitMode, SNAILMode, CavityMode
from quantum_logical.pulsesim.build_hamiltonian import Build_hamiltonian
import matplotlib.pyplot as plt
from quantum_logical.channel import AmplitudeDamping, PhaseDamping
from quantum_logical.trotter import TrotterGroup
from quantum_logical.interaction import ConversionGainInteraction
from qutip import Options
from tqdm import tqdm_notebook as tqdm
from progressbar import progressbar
from threading import Thread
from multiprocessing.pool import Pool, ThreadPool
from numpy.linalg import eig
from quantum_logical.creating_operators import Creating_operators

# opts = Options(nsteps=1e6, atol=1e-8, rtol=1e-6)
# p_bar = qt.ui.TextProgressBar()  # None
opts = {"nsteps": 1e6, "atol": 1e-8, "rtol": 1e-6, "progress_bar": "text"}

In [2]:
fund_ops_list = []
qubit_dim = 3
H = Creating_operators(qubit_count=2, qubit_dim=qubit_dim)
fund_ops_list.append(H.create_ops_unconventional_annhilation(from_level=0, to_level=1))  # this is not necessary with the new addition for the operator 
fund_ops_list.append(H.create_ops_unconventional_annhilation(from_level=1, to_level=2))

# # list of operators based on the fundamental single qubit operators 
# ops_list = []
# ops_list.append(qt.tensor(fund_ops_list[0], fund_ops_list[1].dag()))
# ops_list.append(qt.tensor(fund_ops_list[1], fund_ops_list[0].dag()))
# ops_list.append(qt.tensor(fund_ops_list[1].dag(), fund_ops_list[0]))
# ops_list.append(qt.tensor(fund_ops_list[0].dag(), fund_ops_list[1]))

# list of operators based on the full dimension operators
ops_list = []
ops_list.append(H.creating_ops_full_dim(qubit1_from=1, qubit1_to=0, qubit2_from=1, qubit2_to=2))
ops_list.append(H.creating_ops_full_dim(qubit1_from=2, qubit1_to=1, qubit2_from=0, qubit2_to=1))
ops_list.append(H.creating_ops_full_dim(qubit1_from=1, qubit1_to=2, qubit2_from=1, qubit2_to=0))
ops_list.append(H.creating_ops_full_dim(qubit1_from=0, qubit1_to=1, qubit2_from=2, qubit2_to=1))

# this doesnt work yet but it is necessary for this to work
# ops_list2 = []
# ops_list1 = np.zeros((9,9), dtype=complex)
# for i in range(len(ops_list)):
#     for j in range(ops_list[i].shape[0]):
#         for k in range(ops_list[i].shape[0]):
#             ops_list1[j,k] = ops_list[i][j,k]
#     ops_list2.append(ops_list1)

# for i in range(len(ops_list2)):
#     ops_list2[i] = qt.Qobj(ops_list2[i])

# ops_list[0].shape[0]

In [3]:
qubit1_initial = qt.basis(qubit_dim, 2)
qubit2_initial = qt.basis(qubit_dim, 0)

initial_state = qt.tensor(qubit1_initial, qubit2_initial)

In [4]:
# taking from the f to the e state 
not_gate = H.not_gate(exchange_levels=[1,2])
full_dim_not_gate = qt.tensor(not_gate, qt.qeye(qubit_dim))
state_f_to_e = full_dim_not_gate * initial_state # this is now the state in the e basis 
state_f_to_e

Quantum object: dims=[[3, 3], [1, 1]], shape=(9, 1), type='ket', dtype=Dense
Qobj data =
[[0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]

In [5]:
# apply the cnot via evans method 
H_new = ConversionGainInteraction(gg= np.pi / 4, gc= np.pi / 4, transmon_levels= 3)
U_cnot_gate = H_new.construct_U(1.0)
new_state_after_cnot = U_cnot_gate * state_f_to_e
new_state_after_cnot

Quantum object: dims=[[3, 3], [1, 1]], shape=(9, 1), type='ket', dtype=Dense
Qobj data =
[[0.        +0.j        ]
 [0.        -0.70710678j]
 [0.        +0.j        ]
 [0.70710678+0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]]

In [6]:
# bringing the first qubit back to f
# this does not seem right to me because why is it not in the right final state
final_state_e_f = full_dim_not_gate * new_state_after_cnot
final_state_e_f

Quantum object: dims=[[3, 3], [1, 1]], shape=(9, 1), type='ket', dtype=Dense
Qobj data =
[[0.        +0.j        ]
 [0.        -0.70710678j]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.70710678+0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]]

In [7]:
#ideally the final state should be very similar to the first state but there seems to have been a phase pick up 
np.dot(initial_state.dag(), final_state_e_f)
# this does not seem right

(0.7071067811865476+0j)